In [45]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import re
from datetime import datetime
import numpy as np
import unidecode

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# get all link auchan

In [46]:
def get_all_link_auchan():
    
    offer_page = 'https://www.auchan.hu/ajanlatok'
    
    page = urllib.request.urlopen(offer_page)
    soup = bs(page)
    divs = soup.body.findAll('div', {'class' : ['box submenu']})
        
    all_link = []
    offer_page = 'https://www.auchan.hu/ajanlatok'
    for div in divs:
        for a in div.find_all('a', href=True):
            link = a['href']
            #print(a)
            #print(f'original link:  {link}')
            '''
            if link.endswith('-'):
                link = link[:-1] + '/200'
                all_link.append(link)
            else:
                link = link + '/200'
                all_link.append(link)
            '''
            if link.count('/') == 4:
                link = link + '/mind/500'
                all_link.append(link)
                print(link)
            else:
                link = link + '/300'

    return all_link

In [47]:
all_link = get_all_link_auchan()

https://www.auchan.hu/ajanlat/otthon-haztartas-kert-auto/mind/500
https://www.auchan.hu/ajanlat/sport-jatek/mind/500
https://www.auchan.hu/ajanlat/friss-elelmiszer/mind/500
https://www.auchan.hu/ajanlat/italok/mind/500
https://www.auchan.hu/ajanlat/fagyasztott-elelmiszerek/mind/500
https://www.auchan.hu/ajanlat/tartos-elelmiszerek/mind/500
https://www.auchan.hu/ajanlat/illatszer-szepsegapolas/mind/500
https://www.auchan.hu/ajanlat/allatvilag/mind/500
https://www.auchan.hu/ajanlat/vegyi-aruk-haztartas/mind/500
https://www.auchan.hu/ajanlat/babavilag/mind/500
https://www.auchan.hu/ajanlat/muszaki-cikkek/mind/500
https://www.auchan.hu/ajanlat/ruhazat-1/mind/500


In [48]:
def get_all_offer_auchan(all_link):
    
    def find_nth_occurrence(string, char, occurrence):
        val = -1
        for i in range(0, occurrence):
            val = string.find(char, val + 1)
        return val
    
    counter = 1
    
    all_items = []
    
    for url in all_link:
        
        print(f'crawl url: {url} done {counter} from {len(all_link)}')
        counter = counter + 1
        
        page = urllib.request.urlopen(url)
        soup = bs(page)

        divs = soup.body.findAll('div', {'class' : ['row prodList']})
        
        for div in divs:
            prods = div.findAll('div', {'class' : ['box p10px prodItem']})
            #print(prods)
            if len(prods) > 0:
                for prod in prods:

                    item_dict = {}

                    item_dict['itemId'] = np.nan

                    #print(prod.find('a', {'class' : ['title red']}).get_text())
                    item_dict['itemName'] = prod.find('a', {'class' : ['title red']}).get_text().strip()
                    item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()
                    
                    images = prod.findAll('img')
                    img_url = np.nan
                    for img in images:
                        img_url = img['src']
                        break
                    
                    item_dict['imageUrl'] = img_url

                    #print(prod.find('strong', {'class' : ['red']}).get_text())
                    item_dict['price'] = prod.find('strong', {'class' : ['red']}).get_text().replace('Ft', '').strip()
                    
                    #print(prod.find('p').get_text().strip())
                    
                    #print(item_dict)
                    
                    item_dict['source'] = url[find_nth_occurrence(url, '/', 4)+1:find_nth_occurrence(url, '/', 5)]
                    
                    measure = prod.find('p')
                    
                    if measure != None:
                        measure_cut = measure.get_text().strip('\n').strip('\t').replace('\n','').strip()
                        measure_final = measure_cut[:measure_cut.find(' ')].replace('Kiszerelés:', '')
                        sales_from_string = measure_cut[measure_cut.find(' '):]
                        sales_from_pattern = r'(?P<group_1>[\d]{4}.[\d]{2}.[\d]{2})'
                        sales_from_string = re.search(sales_from_pattern, sales_from_string)[0]
                        item_dict['measure'] = measure_final
                        item_dict['salesStart'] = sales_from_string
                    else:
                        item_dict['measure'] = np.nan
                        
                    item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
                    item_dict['shopName'] = 'auchan'

                    if len(item_dict) != 0:
                        #print('adding item_dict')
                        all_items.append(item_dict)
            else:
                pass
                #print('prod is None')
                        
    df = pd.DataFrame(all_items)
    
    df = df.loc[df['price'] != '']
    
    return df

In [49]:
df = get_all_offer_auchan(all_link)

crawl url: https://www.auchan.hu/ajanlat/otthon-haztartas-kert-auto/mind/500 done 1 from 12
crawl url: https://www.auchan.hu/ajanlat/sport-jatek/mind/500 done 2 from 12
crawl url: https://www.auchan.hu/ajanlat/friss-elelmiszer/mind/500 done 3 from 12
crawl url: https://www.auchan.hu/ajanlat/italok/mind/500 done 4 from 12
crawl url: https://www.auchan.hu/ajanlat/fagyasztott-elelmiszerek/mind/500 done 5 from 12
crawl url: https://www.auchan.hu/ajanlat/tartos-elelmiszerek/mind/500 done 6 from 12
crawl url: https://www.auchan.hu/ajanlat/illatszer-szepsegapolas/mind/500 done 7 from 12
crawl url: https://www.auchan.hu/ajanlat/allatvilag/mind/500 done 8 from 12
crawl url: https://www.auchan.hu/ajanlat/vegyi-aruk-haztartas/mind/500 done 9 from 12
crawl url: https://www.auchan.hu/ajanlat/babavilag/mind/500 done 10 from 12
crawl url: https://www.auchan.hu/ajanlat/muszaki-cikkek/mind/500 done 11 from 12
crawl url: https://www.auchan.hu/ajanlat/ruhazat-1/mind/500 done 12 from 12


In [50]:
df

,itemId,itemName,itemCleanName,imageUrl,price,source,measure,salesStart,runDate,shopName
0,NaN,ACTUEL 2 RÉSZES ÁGYNEMŰHUZAT- GARNITÚRA,actuel 2 reszes agynemuhuzat- garnitura,https://www.auchan.hu/userfiles//b/a/baad947f6...,3990,otthon-haztartas-kert-auto,Ft/garnitúra,2022.07.07,2022.07.11-10:00:58,auchan
2,NaN,ACTUEL KISPÁRNAHUZAT,actuel kisparnahuzat,https://www.auchan.hu/userfiles//b/6/b600dbe38...,799,otthon-haztartas-kert-auto,Ft/db,2022.07.07,2022.07.11-10:00:58,auchan
3,NaN,ACTUEL 5 RÉSZES DUPLA PAPLANOS ÁGYNEMŰH...,actuel 5 reszes dupla paplanos agynemuh...,https://www.auchan.hu/userfiles//6/0/601d36b75...,8990,otthon-haztartas-kert-auto,Ft/garnitúra,2022.07.07,2022.07.11-10:00:58,auchan
4,NaN,AUCHAN OPTIMUM 3 RÉSZES ÁGYNEMŰHUZAT- G...,auchan optimum 3 reszes agynemuhuzat- g...,https://www.auchan.hu/userfiles//2/f/2f60b3c96...,3990,otthon-haztartas-kert-auto,Ft/garnitúra,2022.07.07,2022.07.11-10:00:58,auchan
5,NaN,NATURTEX HOME ALOE VERA KISPÁRNA,naturtex home aloe vera kisparna,https://www.auchan.hu/userfiles//4/0/40648d63d...,2990,otthon-haztartas-kert-auto,Ft/db,2022.07.07,2022.07.11-10:00:58,auchan
...,...,...,...,...,...,...,...,...,...,...
549,NaN,FÉRFI MINTÁS PÓLÓ,ferfi mintas polo,https://www.auchan.hu/userfiles//7/8/780482dcb...,1690,ruhazat-1,Ft/db,2022.07.07,2022.07.11-10:01:22,auchan
550,NaN,FÉRFI SZÖRFNADRÁG,ferfi szorfnadrag,https://www.auchan.hu/userfiles//4/4/4471671ed...,2590,ruhazat-1,Ft/db,2022.07.07,2022.07.11-10:01:22,auchan
551,NaN,GYERMEK PÓLÓ,gyermek polo,https://www.auchan.hu/userfiles//9/c/9cb5c4330...,799,ruhazat-1,Ft/db,2022.07.07,2022.07.11-10:01:22,auchan
552,NaN,2 DB-OS BABA BODY,2 db-os baba body,https://www.auchan.hu/userfiles//3/8/38120d5cc...,1590,ruhazat-1,Ft/db,2022.07.07,2022.07.11-10:01:22,auchan


In [51]:
df = df.fillna('N.a')

In [52]:
df.loc[~df['price'].str.isnumeric()]

,itemId,itemName,itemCleanName,imageUrl,price,source,measure,salesStart,runDate,shopName


In [53]:
df[df['imageUrl'].isnull()]

,itemId,itemName,itemCleanName,imageUrl,price,source,measure,salesStart,runDate,shopName


In [54]:
df.to_excel('auchan_result.xlsx', index=False)

# backup

In [54]:
url = 'https://www.auchan.hu/ajanlat/otthon-haztartas-kert-auto/200'

In [55]:
page = urllib.request.urlopen(url)
soup = bs(page)

In [56]:
divs = soup.body.findAll('div', {'class' : ['row prodList']})

In [57]:
len(divs)

1

In [58]:
for div in divs:
    prods = div.findAll('div', {'class' : ['box p10px prodItem']})
    for prod in prods:
        print(prod.find('a', {'class' : ['title red']}).get_text())
        print(prod.find('strong', {'class' : ['red']}).get_text())
        print(prod.find('p').get_text().strip())

 ACTUEL CSÚSZÁSGÁTLÓ
1490 Ft
Kiszerelés:Ft/db
                2022.06.23-06.29.
 ACTUEL MŰANYAG WC-ÜLŐKE
4490 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 SZÁMOS ACTUEL FÜRDŐSZOBASZŐNYEG - 20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.
 SZÁMOS ACTUEL ZUHANYFÜGGÖNY - 20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.
 ACTUEl BÁRSONY STRAND- TÖRÖLKÖZŐ
6990 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEL MICROFIBER TÖRÖLKÖZŐ
1790 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEl BÁRSONY STRANDTÖRÖLKÖZŐ
3690 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEl STRANDMATRAC PÁRNÁVAL
6490 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 ACTUEl ITALADAGOLÓ
999 Ft
Kiszerelés:Ft/db
            2022.06.23-06.29.
 SZÁMOS BOROS ÉS PEZSGŐS POHÁR -20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.
 SZÁMOS KANCSÓ -20 %

Kiszerelés:Ft/db
            2022.06.23-06.29.
 SZÁMOS TÁL   -20 %

Kiszerelés:Ft/db
                2022.06.23-06.29.


In [ ]:
def get_all_link_auchan():
    
    all_link = []
    offer_page = 'https://www.auchan.hu/ajanlatok'
    for div in divs:
        for a in div.find_all('a', href=True):
            print(a['href'])
            all_link.append(a['href'] + '/500')
    
    return all_link